In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

from scipy import interpolate

In [ ]:
file_path = './data/rainfall_train_v2_vvO.csv'
rainfall = pd.read_csv(file_path)
rainfall = rainfall.drop(['fc_year'], axis=1) #, 'ef_year', 'stn4contest'
rainfall

In [16]:
rainfall.columns

Index(['fc_month', 'fc_day', 'fc_hour', 'dh', 'ef_month', 'ef_day', 'ef_hour',
       'v01', 'v02', 'v03', 'v04', 'v05', 'v06', 'v07', 'v08', 'v09', 'vv',
       'class_interval', 'quarter', 'fc_day_y', 'rng_v01', 'rng_v02',
       'rng_v03', 'rng_v04', 'rng_v05', 'rng_v06', 'rng_v07', 'rng_v08',
       'rng_v09', 'pred', 'timestamp_log'],
      dtype='object')

In [17]:
with open('./function/requirements_import.py') as f:
    exec(f.read())
    
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import *
from tensorflow.keras import layers, models, Model, Input

es = EarlyStopping(monitor='val_loss',        # 얼리스토핑을 적용할 관측 지표
                min_delta=0,               # 임계값
                patience=5,                # 성능 개선이 발생하지 않았을 때, 몇 epochs 더 지켜볼 것인지
                verbose=0,                 # 몇 번째 epochs에서 얼리스토핑이 되었는가 알려줌
                restore_best_weights=True  # 최적의 가중치를 가진 epoch 시점으로 가중치를 되돌림
                )

In [19]:
# 종속변수
target = ['vv', 'class_interval']

# x, y
x = rainfall.drop(columns=target)
y_vv = rainfall.loc[:, target[0]]
y_cl = rainfall.loc[:, target[1]].astype(int)

aa, bb = idx_find(x)

# 7:1.7:1.3 데이터로 분할
x_train = x.iloc[:aa+1]
y_train = y_vv.iloc[:aa+1]

x_val = x.iloc[aa+1:bb+1]
y_val = y_vv.iloc[aa+1:bb+1]

x_test = x.iloc[bb+1:]
y_test = y_vv.iloc[bb+1:]
print(x_train.shape, x_val.shape, x_test.shape)
print(y_train.shape, y_val.shape, y_test.shape)


# # GAF : 시계열 -> 이미지
# gaf = GramianAngularField(method='summation')
# x_gaf = gaf.fit_transform(x)
# x_gaf4 = x_gaf.reshape(-1, x.shape[1], x.shape[1], 1)

# # 각 범주의 최소 데이터 수 찾기
# ind_class = [np.where(y == i)[0] for i in range(10)]
# min_samples = min(len(indices) for indices in ind_class)

# # 각 범주에서 최소 데이터 수만큼 랜덤 샘플 추출
# bal_ind = np.hstack([np.random.choice(indices, min_samples, replace=False) for indices in ind_class])

# # 추출한 인덱스로 데이터와 레이블 준비
# bal_x = x_gaf4[bal_ind]
# bal_y = y[bal_ind]

# # 7:1.7:1.3 데이터로 분할
# x_train, x_test, y_train, y_test = train_test_split(bal_x, bal_y, test_size=0.3, random_state=42)
# x_val, x_test, y_val, y_test = train_test_split(x_test, y_test, test_size=0.4, random_state=42)

# print(x_train.shape, x_val.shape, x_test.shape)
# print(y_train.shape, y_val.shape, y_test.shape)

(871520, 29) (363120, 29) (218080, 29)
(871520,) (363120,) (218080,)


In [27]:
clear_session()

from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
x_train = ss.fit_transform(x_train)
x_val = ss.transform(x_val)
x_test = ss.transform(x_test)

model = Sequential([
    Dense(29, input_shape=(29,), activation='relu'),
    # Dropout(0.5),
    BatchNormalization(),
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dense(16, activation='relu'),
    BatchNormalization(),
    Dense(8, activation='relu'),
    BatchNormalization(),
    Dense(1)
])

model.summary()

# 모델 설정
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mse')


# lr_sch = LearningRateScheduler(scheduler)

# 학습
history = model.fit(x_train, y_train, epochs = 30, callbacks=[es], validation_data=(x_val, y_val)).history #lr_sch, 

# 예측
pred = model.predict(x_test)
print(f'RMSE : {mean_squared_error(y_test, pred, squared=False)}')
print(f'MAE  : {mean_absolute_error(y_test, pred)}')
print(f'MAPE : {mean_absolute_percentage_error(y_test, pred)}')

# beepsound()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 29)             │           870 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 29)             │           116 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 16)             │           480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 16)             │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 8)              │            32 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │             9 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,043 (7.98 KB)

 Trainable params: 1,905 (7.44 KB)

 Non-trainable params: 138 (552.00 B)

Epoch 1/30
27235/27235 ━━━━━━━━━━━━━━━━━━━━ 114s 4ms/step - loss: 4535.6963 - val_loss: 7201.5723
Epoch 2/30
15146/27235 ━━━━━━━━━━━━━━━━━━━━ 58s 5ms/step - loss: 4526.5894

KeyboardInterrupt: 

In [ ]:
clear_session()

cnn_model = models.Sequential([Rescaling(1/255, input_shape=x_train_res.shape[1:]), # input_shape=(32, 32, 3)
                    Conv2D(29, kernel_size = 3, strides=(1,1), padding='same', activation='relu'),
                    MaxPooling2D(pool_size = 2 ),
                    Conv2D(64, kernel_size = 3, strides=(1,1), padding='same', activation='relu'),
                    MaxPooling2D(pool_size = 2 ),
                    Flatten()])

# 입력 정의
input_layer = Input(shape=x_train_res.shape[1:])

# CNN 모델을 통해 특징 추출
cnn_features = cnn_model(input_layer)

# CNN 출력 reshape
output_shape = cnn_model.output_shape[-1]
cnn_features_reshaped = layers.Reshape((1, output_shape))(cnn_features)  # GRU 입력 형태에 맞추기 위해 reshape

# GRU 모델을 통해 분류
gru_model = models.Sequential([
    layers.GRU(32, return_sequences=True, input_shape=(None, 3136)),
    layers.GRU(16),
    layers.Dense(10, activation='softmax')  # 이진 분류 예제
])

output_layer = gru_model(cnn_features_reshaped)

# 전체 모델 정의
model = Model(inputs=input_layer, outputs=output_layer)

In [ ]:
model.summary()

# 모델 설정
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy', csi_metric])

es = EarlyStopping(monitor='val_loss',        # 얼리스토핑을 적용할 관측 지표
                min_delta=0,               # 임계값
                patience=5,                # 성능 개선이 발생하지 않았을 때, 몇 epochs 더 지켜볼 것인지
                verbose=0,                 # 몇 번째 epochs에서 얼리스토핑이 되었는가 알려줌
                restore_best_weights=True  # 최적의 가중치를 가진 epoch 시점으로 가중치를 되돌림
                )
# 학습
history = model.fit(x_train_res, y_train_res, epochs = 300, callbacks=[es], validation_data=(x_val, y_val), batch_size=64).history #lr_sch, 

# 예측
pred = model.predict(x_test)
pred_1 = pred.argmax(axis=1)

cm = confusion_matrix(y_test, pred_1)
print(csi(cm))
print('-'*60)
print(cm)
print('-'*60)
print(classification_report(y_test, pred_1))

beepsound()

In [ ]:
### 학습곡선 (loss)
plt.figure(figsize=(10,6))
plt.subplot(1,2,1)
plt.plot(history['loss'], label='train_err', marker = ',')
plt.plot(history['val_loss'], label='val_err', marker = 'o')

plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.title('Loss')

### 학습곡선 (Accuracy)
plt.subplot(1,2,2)
plt.plot(history['accuracy'], label='train_acc', marker = ',')
plt.plot(history['val_accuracy'], label='val_acc', marker = 'o')

plt.ylabel('accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.title('Accuracy')

plt.tight_layout()
plt.show()

# 변수정리

In [ ]:
temp

In [ ]:
# temp = rainfall.copy()
temp3 = temp.drop(columns= ['fc_month', 'ef_month', 'fc_day_y'] + [f'roll_v0{i}' for i in range(1,6)] + [f'rng_v0{i}' for i in range(8,10)] + [f'roll_rng_v0{i}' for i in range(6,8)])

temp2 = temp3.corr()

mask = np.tril(np.ones_like(temp2, dtype=bool))

plt.figure(figsize=(20,20))
sns.heatmap(temp2, mask=mask, vmin=-1, vmax=1, annot=True, fmt=".2f", cmap="YlGnBu")

In [ ]:
target = 'vv'
temp4 = temp3.drop(columns=['ef_hour', 'timestamp'])
x = temp4.drop(target, axis=1)
y = temp4.loc[:, target]

train_input = x.iloc[:866993]
test_input = x.iloc[866993:]
train_target = y.iloc[:866993]
test_target = y.iloc[866993:]

In [ ]:
xgb = XGBRegressor(objective='reg:squarederror', eval_metric='rmse')
xgb.fit(train_input, train_target)
y_pred = xgb.predict(test_input)

# 평가
mse = mean_squared_error(test_target, y_pred)
r2 = r2_score(test_target, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

In [ ]:
plt.figure(figsize=(15, 35))  # 그림 크기 조정
plot_importance(xgb, importance_type='weight')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(train_input, train_target)
y_pred = rf.predict(test_input)

# 평가
mse = mean_squared_error(test_target, y_pred)
r2 = r2_score(test_target, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gbm = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
gbm.fit(train_input, train_target)
y_pred = gbm.predict(test_input)

# 정확도 평가
accuracy = accuracy_score(test_target, y_pred)
print("Accuracy:", accuracy)

# 혼동 행렬 생성
cm = confusion_matrix(test_target, y_pred)

# 시각화
fig, ax = plt.subplots(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gbm.classes_)
disp.plot(ax=ax)
plt.show()

# 튜닝

In [ ]:
params = {
    'learning_rate': [0.005, 0.01, 0.1, 0.3],  # 학습률 (다양한 값 추가)
    'n_estimators': [100, 300, 500],  # 트리 개수 (다양한 값 추가)
    'max_depth': [5, 10, 20],  # 트리 최대 깊이 (다양한 값 추가)
    'min_child_weight': [1, 5, 20],  # 최소 자식 노드 가중치 (다양한 값 추가)
    'gamma': [0, 0.1, 0.2, 0.3],  # 가중치에 대한 페널티 (다양한 값 추가)
    'subsample': [0.8, 0.9, 1.0],  # 트리에 사용할 샘플 비율 (다양한 값 추가)
    'colsample_bytree': [0.8, 0.9, 1.0]  # 각 트리에 사용할 특성 비율 (다양한 값 추가)
}
# params = {
#     'learning_rate': [0.001, 0.005, 0.01, 0.1, 0.2, 0.3],  # 학습률 (다양한 값 추가)
#     'n_estimators': [100, 200, 300],  # 트리 개수 (다양한 값 추가)
#     'max_depth': [3, 4, 5, 6, 7, 8, 9],  # 트리 최대 깊이 (다양한 값 추가)
#     'min_child_weight': [1, 5, 10, 20],  # 최소 자식 노드 가중치 (다양한 값 추가)
#     'gamma': [0, 0.1, 0.2, 0.3],  # 가중치에 대한 페널티 (다양한 값 추가)
#     'subsample': [0.8, 0.9, 1.0],  # 트리에 사용할 샘플 비율 (다양한 값 추가)
#     'colsample_bytree': [0.8, 0.9, 1.0]  # 각 트리에 사용할 특성 비율 (다양한 값 추가)
# }

# 모델생성
xgb = XGBClassifier(random_state=42)

# AutoML모델 생성
gs = GridSearchCV(xgb, param_grid=params, n_jobs=-1, scoring='accuracy', cv=5)

# AutoML모델 훈련
gs.fit(train_input, train_target)

# 찾은 최적모델 사용
xgb = gs.best_estimator_
y_pred = xgb.predict(test_input)

accuracy = accuracy_score(test_target, y_pred)
print("Accuracy:", accuracy)
# 혼동 행렬 생성
cm = confusion_matrix(test_target, y_pred)

# 시각화
fig, ax = plt.subplots(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=xgb.classes_)
disp.plot(ax=ax)
plt.show()

In [ ]:
xgb = XGBClassifier(max_depth= ,n_extimators=100)
xgb.fit(train_input, train_target)
y_pred = xgb.predict(test_input)

# 정확도 평가
accuracy = accuracy_score(test_target, y_pred)
print("Accuracy:", accuracy)

# 혼동 행렬 생성
cm = confusion_matrix(test_target, y_pred)

# 시각화
fig, ax = plt.subplots(figsize=(10, 8))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=xgb.classes_)
disp.plot(ax=ax)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split


print(df['target'].value_counts())


In [ ]:
# 예제 데이터 생성
data = {
    'datetime': pd.date_range(start='2023-05-01', periods=100, freq='H'),
    'feature1': np.random.randn(100),
    'target': np.random.randint(0, 10, size=100)
}

df = pd.DataFrame(data)
# datetime을 인덱스로 설정
df.set_index('datetime', inplace=True)

x = df.drop(columns=['target'])
y = df['target']

X_train = x.iloc[:70]
X_test = x.iloc[70:]
y_train = y.iloc[:70]
y_test = y.iloc[70:]

# SMOTE 적용
smote = SMOTE(sampling_strategy=0.3, random_state=42, k_neighbors=1)
X_res, y_res = smote.fit_resample(X_train, y_train)

# 업샘플링된 데이터프레임 생성
df_resampled = pd.DataFrame(X_res, columns=x.columns)
df_resampled['target'] = y_res

print(df_resampled['target'].value_counts())


In [ ]:
df

In [ ]:
df_resampled